In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

In [2]:
conn = sqlite3.connect(r"c:\Dev\06. Data\3. FDA NCTs - SQL\studies_db_06-Nov-2018_all date fields as ints.sqlite3")
df = pd.read_sql_query("SELECT * FROM location_countries", conn)
df.head()

,id,nct_id,country
0,1,NCT00000102,United States of America
1,2,NCT00000104,United States of America
2,3,NCT00000105,United States of America
3,4,NCT00000106,United States of America
4,5,NCT00000107,United States of America


In [3]:
df_countries_list = pd.read_sql_query("SELECT DISTINCT country FROM location_countries ORDER BY country ASC", conn)
df_countries_list.to_csv(r"c:\Dev\06. Data\3. FDA NCTs - SQL\countries_from_sql.csv")
df_countries_list.head()

,country
0,**None**
1,Afghanistan
2,Albania
3,Algeria
4,American Samoa


In [4]:
country_phase_by_year = pd.read_sql_query("""
    SELECT Count(s1.nct_id) as nct_id, strftime('%Y', datetime(s1.study_first_submitted, 'unixepoch')) as year
            FROM studies as s1
            WHERE s1.phase = 'Phase 1' or s1.phase = 'Early Phase 1'
            GROUP BY strftime('%Y', datetime(s1.study_first_submitted, 'unixepoch'))  
            ORDER BY year ASC
""", conn)
country_phase_by_year.head()

,nct_id,year
0,684,1999
1,255,2000
2,231,2001
3,181,2002
4,233,2003


In [5]:
country_phase_by_year.index = country_phase_by_year.year
country_phase_by_year.drop(columns=["year"], axis=1, inplace=True)
country_phase_by_year.head()

,nct_id
year,
1999,684
2000,255
2001,231
2002,181
2003,233


In [6]:
print(country_phase_by_year.to_dict())

{'nct_id': {'1999': 684, '2000': 255, '2001': 231, '2002': 181, '2003': 233, '2004': 247, '2005': 844, '2006': 1011, '2007': 1514, '2008': 2179, '2009': 2460, '2010': 2300, '2011': 2398, '2012': 2329, '2013': 2218, '2014': 3019, '2015': 2437, '2016': 2493, '2017': 2418, '2018': 2015}}


In [9]:
conn.execute("""
    CREATE VIEW IF NOT EXISTS counted_countries AS 
        SELECT country, phase, strftime('%Y', datetime(s.study_first_submitted, 'unixepoch')) as year, 
                COUNT(nct_id) as id_count 
        FROM (
            SELECT location_countries.country as country, stud.* 
            FROM studies as stud 
        LEFT JOIN location_countries ON location_countries.nct_id = stud.nct_id ) as s
        GROUP BY country, phase, strftime('%Y', datetime(s.study_first_submitted, 'unixepoch')) 
        ORDER BY country, phase, strftime('%Y', datetime(s.study_first_submitted, 'unixepoch'))
""")

OperationalError: database is locked

In [11]:
df_studies_phases_by_year = pd.read_sql_query("""
    SELECT counted_countries.country, counted_countries.year, COALESCE(SUM(phase1), 0) as phase1, 
        COALESCE(SUM(phase2), 0) as phase2, COALESCE(SUM(phase3), 0) as phase3, 
        COALESCE(SUM(phase4), 0) as phase4 FROM counted_countries 
    LEFT JOIN ( 
        SELECT country, phase, year, SUM(id_count) as phase1 
        FROM counted_countries 
        WHERE phase = "Phase 1" GROUP BY country, phase, year ) as phase1Tbl 
    ON counted_countries.country = phase1Tbl.country and counted_countries.phase = phase1Tbl.phase and counted_countries.year = phase1Tbl.year 
    
    LEFT JOIN ( 
        SELECT country, phase, year, SUM(id_count) as phase2 
        FROM counted_countries 
        WHERE phase = "Phase 2" GROUP BY country, phase, year ) as phase2Tbl 
    ON counted_countries.country = phase2Tbl.country and counted_countries.phase = phase2Tbl.phase and counted_countries.year = phase2Tbl.year 
    
    LEFT JOIN ( 
        SELECT country, phase, year, SUM(id_count) as phase3 
        FROM counted_countries 
        WHERE phase = "Phase 3" GROUP BY country, phase, year ) as phase3Tbl 
    ON counted_countries.country = phase3Tbl.country and counted_countries.phase = phase3Tbl.phase and counted_countries.year = phase3Tbl.year 
    
    LEFT JOIN ( 
        SELECT country, phase, year, SUM(id_count) as phase4 
        FROM counted_countries 
        WHERE phase = "Phase 4" GROUP BY country, phase, year ) as phase4Tbl 
    ON counted_countries.country = phase4Tbl.country and counted_countries.phase = phase4Tbl.phase and counted_countries.year = phase4Tbl.year

    WHERE counted_countries.country = "Argentina" GROUP BY counted_countries.country, counted_countries.year
    """, conn)
df_studies_phases_by_year.head()

,country,year,phase1,phase2,phase3,phase4
0,Argentina,1999,0,5,4,0
1,Argentina,2000,0,2,4,0
2,Argentina,2001,0,2,4,0
3,Argentina,2002,0,2,16,3
4,Argentina,2003,0,7,14,0


In [ ]:
df_studies_phases_by_year.index = df_studies_phases_by_year.month
print(df_studies_phases_by_year.to_dict(orient="index"))

In [ ]:
PHASES_DICT = {
    "unknown" : ["**None**", "N/A"],
    "phase1": ["Phase 1", "Early Phase 1"],
    "phase2": ["Phase 2", "Phase 1/Phase 2"],
    "phase3": ["Phase 3", "Phase 2/Phase 3"],
    "phase4": ["Phase 4"]
}
cond_preamb = "s1.phase = '"
cond_concat = " or "
for country in df_countries_list.country:
    print("Country:{}".format(country))
    for phase in PHASES_DICT:
        where_condition = ""
        phase_condition_array = PHASES_DICT[phase]
        i=0
        while True:
            where_condition += cond_preamb+phase_condition_array[i]+"'"
            i+=1
            if i<len(phase_condition_array):
                where_condition += cond_concat
                continue
            else: break
        print("Phase:{}    Condition:{}".format(phase, where_condition))
        
    

Returns all records (phase, year by country)

SELECT phase, strftime('%Y', datetime(s.study_first_submitted, 'unixepoch')) as year, COUNT(nct_id)
    FROM 
	(
		SELECT location_countries.country as country, stud.*
		FROM studies as  stud
		LEFT JOIN location_countries ON location_countries.nct_id = stud.nct_id
		WHERE location_countries.country = "Argentina"
	)
	as s
 GROUP BY phase, strftime('%Y', datetime(s.study_first_submitted, 'unixepoch'))
 ORDER BY phase, strftime('%Y', datetime(s.study_first_submitted, 'unixepoch'))
 


view

CREATE VIEW counted_countries
AS
SELECT country, phase, strftime('%Y', datetime(s.study_first_submitted, 'unixepoch')) as year, COUNT(nct_id) as id_count
		FROM 
		(
			SELECT location_countries.country as country, stud.*
			FROM studies as  stud
			LEFT JOIN location_countries ON location_countries.nct_id = stud.nct_id
		)
		as s
	 GROUP BY country, phase, strftime('%Y', datetime(s.study_first_submitted, 'unixepoch'))
	 ORDER BY country, phase, strftime('%Y', datetime(s.study_first_submitted, 'unixepoch'))

GOOD SQL to build table

SELECT counted_countries.country, counted_countries.year, 
		COALESCE(SUM(phase1), 0) as phase1, COALESCE(SUM(phase2), 0) as phase2, 
		COALESCE(SUM(phase3), 0) as phase3, COALESCE(SUM(phase4), 0) as phase4
FROM counted_countries
LEFT JOIN (
	SELECT country, phase, year, SUM(id_count) as phase1
	FROM counted_countries
	WHERE phase = "Phase 1"
	GROUP BY country, phase, year
) as phase1Tbl
  ON counted_countries.country = phase1Tbl.country and counted_countries.phase = phase1Tbl.phase and counted_countries.year = phase1Tbl.year
LEFT JOIN (
	SELECT country, phase, year, SUM(id_count) as phase2
	FROM counted_countries
	WHERE phase = "Phase 2"
	GROUP BY country, phase, year
) as phase2Tbl
  ON counted_countries.country = phase2Tbl.country and counted_countries.phase = phase2Tbl.phase and counted_countries.year = phase2Tbl.year
LEFT JOIN (
	SELECT country, phase, year, SUM(id_count) as phase3
	FROM counted_countries
	WHERE phase = "Phase 3"
	GROUP BY country, phase, year
) as phase3Tbl
  ON counted_countries.country = phase3Tbl.country and counted_countries.phase = phase3Tbl.phase and counted_countries.year = phase3Tbl.year
LEFT JOIN (
	SELECT country, phase, year, SUM(id_count) as phase4
	FROM counted_countries
	WHERE phase = "Phase 4"
	GROUP BY country, phase, year
) as phase4Tbl
  ON counted_countries.country = phase4Tbl.country and counted_countries.phase = phase4Tbl.phase and counted_countries.year = phase4Tbl.year

  
WHERE counted_countries.country = "Argentina"
GROUP BY counted_countries.country, counted_countries.year